# E-commerce Data Analysis for Data Analytics Interview

This notebook is a complete case study of an end-to-end analysis on an e-commerce dataset, from data cleaning to the presentation of actionable insights.

**Dataset:** Contains transactions from a UK online store between 01/12/2010 and 09/12/2011.

In [28]:
import pandas as pd
import numpy as np
import plotly.express as px

## 1. Data Configuration and Loading

In [2]:
df = pd.read_csv('../data/data.csv', encoding='latin1')
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


## 2. Data Cleaning

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


- `Description` and `CustomerID` have null values.
- `InvoiceDate` is an `object` (string) and needs to be converted to `datetime`.
- `CustomerID` is a `float`, but should be an `int` or `string` (identifier).

Null values ​​in `CustomerID` are problematic because I cannot assign transactions to a specific customer. Since repeat customer analysis is one of the goals.

In [4]:
# Checking the percentage of null values
print(f"Percentage of null CustomerIDs: {df['CustomerID'].isnull().sum() * 100 / len(df):.2f}%")

Percentage of null CustomerIDs: 24.93%


Decision: Remove rows where CustomerID is null.
The quantity is significant (24.93%), but essential for customer analysis.

In [5]:
df.dropna(subset=['CustomerID'], inplace=True)

In [6]:
# Checking the percentage of null values
print(f"Percentage of null CustomerIDs: {df['CustomerID'].isnull().sum() * 100 / len(df):.2f}%")

Percentage of null CustomerIDs: 0.00%


In [7]:
# Convert the column, turning any problematic dates into NaT
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'], errors='coerce')
invalid_dates = df[df['InvoiceDate'].isna()]
print(f"Found {len(invalid_dates)} rows with invalid dates.")


Found 0 rows with invalid dates.


In [8]:
df['CustomerID'] = df['CustomerID'].astype(str)

#### Processing Invalid Data

In [9]:
# Verificando estatísticas descritivas para encontrar anomalias
df.describe()

,Quantity,InvoiceDate,UnitPrice
count,406829.000000,406829,406829.000000
mean,12.061303,2011-07-10 16:30:57.879207424,3.460471
min,-80995.000000,2010-12-01 08:26:00,0.000000
25%,2.000000,2011-04-06 15:02:00,1.250000
50%,5.000000,2011-07-31 11:48:00,1.950000
75%,12.000000,2011-10-20 13:06:00,3.750000
max,80995.000000,2011-12-09 12:50:00,38970.000000
std,248.693370,NaN,69.315162


- `Quantity` has negative minimum values. This usually represents returns.

- `UnitPrice` has a minimum value of 0.0. Selling something at zero price may be a registration error or a promotion, but for revenue analysis, these items do not add value and can distort metrics such as average price.

In [10]:
print(f"Percentage of negative quantity: {(df['Quantity'] < 0).sum() * 100 / len(df):.2f}%")

Percentage of negative quantity: 2.19%


In [11]:
# Filtering the DataFrame to keep only sales transactions (Quantity > 0)
df = df[df['Quantity'] > 0]

In [12]:
print(f"Percentage of items with price of zero: {(df['UnitPrice'] == 0).sum() * 100 / len(df):.2f}%")

Percentage of items with price of zero: 0.01%


In [13]:
# Filtering to remove items with a price of zero, which do not contribute to revenue
df = df[df['UnitPrice'] > 0]

In [ ]:
# Checking the DataFrame dimensions after cleaning
print("DataFrame dimensions after cleaning:", df.shape)

DataFrame dimensions after cleaning: (397884, 8)


## 3. Feature Engineering

In [17]:
# Creating the 'TotalPrice' column
df['TotalPrice'] = df['Quantity'] * df['UnitPrice']

In [18]:
# Extracting date information for temporal analysis
df['MonthYear'] = df['InvoiceDate'].dt.to_period('M').astype(str)

In [19]:
df.head(1)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalPrice,MonthYear
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.3,2010-12


## 4. Exploratory Data Analysis (EDA)

### 4.1. Sales Analysis by Period

What is the sales trend over time? Is there any seasonality?

In [21]:
# Grouping sales by month
monthly_sales = df.groupby('MonthYear')['TotalPrice'].sum().reset_index()
monthly_sales = monthly_sales.sort_values('MonthYear')
monthly_sales

,MonthYear,TotalPrice
0,2010-12,572713.890
1,2011-01,569445.040
2,2011-02,447137.350
3,2011-03,595500.760
4,2011-04,469200.361
5,2011-05,678594.560
6,2011-06,661213.690
7,2011-07,600091.011
8,2011-08,645343.900
9,2011-09,952838.382


In [31]:
fig = px.line(monthly_sales, x='MonthYear', y='TotalPrice',
              title='Total Monthly Sales (2011)',
              labels={'MonthYear': 'Month', 'TotalPrice': 'Total Recipe'},
              template='plotly_dark' 
             )


fig.update_layout(
    font_family="Arial, sans-serif",
    title_font_size=24,
    xaxis_title_font_size=16,
    yaxis_title_font_size=16,
    hovermode="x unified", 
    margin=dict(l=40, r=40, t=80, b=40), 
    plot_bgcolor='rgba(0,0,0,0)',
    paper_bgcolor='rgba(0,0,0,0)', 
    xaxis=dict(
        showgrid=True,
        gridcolor='rgba(255,255,255,0.1)', 
        linecolor='rgba(255,255,255,0.2)',
        tickfont_size=12
    ),
    yaxis=dict(
        showgrid=True,
        gridcolor='rgba(255,255,255,0.1)', 
        linecolor='rgba(255,255,255,0.2)',
        tickfont_size=12
    )
)

fig.update_traces(
    line=dict(width=3, color='#636EFA'), 
    mode='lines+markers', 
    marker=dict(size=8, symbol='circle', line=dict(width=1, color='DarkSlateGrey'))
)

fig.show()

**Analysis of the Potential Drop:**

The graph shows a **sharp drop in December 2011**.

In [32]:
print(f"Last date in dataset: {df['InvoiceDate'].max()}")

Last date in dataset: 2011-12-09 12:50:00


**Conclusion regarding the drop:** The "drop" in December is explained by the fact that **there is only data up to the 9th**. It's not a performance drop, but rather an artifact of incomplete data for the month. This is a crucial insight that prevents misguided business conclusions.

### 4.2. Sales Analysis by Category (Product)

In [34]:
# Grouping by product description
top_products = df.groupby('Description')['TotalPrice'].sum().nlargest(10).reset_index()

In [35]:
import plotly.express as px

fig = px.bar(top_products, 
             x='TotalPrice', 
             y='Description',
             orientation='h', 
             title='Top 10 Products by Revenue',
             labels={'TotalPrice': 'Total Revenue', 'Description': 'Product'},
             template='plotly_dark',
             text='TotalPrice'
            )

fig.update_layout(
    font_family="Arial, sans-serif",
    title_font_size=24,
    xaxis_title_font_size=16,
    yaxis_title_font_size=16,
    margin=dict(l=40, r=40, t=80, b=40),
    plot_bgcolor='rgba(0,0,0,0)',
    paper_bgcolor='rgba(0,0,0,0)',
    xaxis=dict(
        showgrid=True,
        gridcolor='rgba(255,255,255,0.1)',
        linecolor='rgba(255,255,255,0.2)',
        tickfont_size=12
    ),
    yaxis=dict(
        categoryorder='total ascending',
        showgrid=False,
        linecolor='rgba(255,255,255,0.2)',
        tickfont_size=12
    )
)

fig.update_traces(
    marker_color='#17A2B8',
    texttemplate='%{text:,.0f}',
    textposition='inside'
)

fig.show()


### 4.3. Customer Analysis: Recurring vs. New Customers

In [43]:
customer_invoices = df.groupby('CustomerID')['InvoiceNo'].nunique().reset_index()
customer_invoices.rename(columns={'InvoiceNo': 'TotalInvoices'}, inplace=True)
customer_invoices['CustomerType'] = np.where(customer_invoices['TotalInvoices'] > 1, 'Recurring', 'New')
customer_type_counts = customer_invoices['CustomerType'].value_counts().reset_index()

In [45]:
import plotly.express as px

fig = px.pie(customer_type_counts, 
             names='CustomerType', 
             values='count', 
             title='Customer Proportion: Recurrings vs. News',
             labels={'CustomerType': 'Tipo de Cliente', 'count': 'Número de Clientes'},
             template='plotly_dark',
             hole=.4
            )

fig.update_layout(
    font_family="Arial, sans-serif",
    title_font_size=24,
    legend_title_font_size=14,
    legend_font_size=12,
    margin=dict(l=40, r=40, t=80, b=40),
    paper_bgcolor='rgba(0,0,0,0)',
)

fig.update_traces(
    textposition='inside',
    textinfo='percent+label',
    insidetextfont=dict(color='white', size=14),
    marker=dict(colors=['#636EFA', '#00CC96'], line=dict(color='#111111', width=2))
)

fig.show()


**Analysis:** The majority of customers (67.4%) made more than one purchase during the period, indicating a strong base of repeat customers. Retention appears to be a strength of this business.

## 5. Conclusion

**Insight 1: The "drop" in December sales is a false alarm.**
- **Note:** December 2011 sales appear drastically lower, but analysis revealed that the data only covers the first 9 days of the month.

- **Action:** When reporting, it is crucial to contextualize this information to prevent leadership from making decisions based on a false premise of declining performance. Recommend obtaining complete data for future year-end analyses.

**Insight 2:** The business has a loyal customer base, but with high-value customers concentrated.
- **Note:** More than 2/3 of customers are recurring customers. A deeper analysis (not included here, but a next step) would likely show that a small fraction of these customers (Pareto principle) generates the majority of the revenue.

- **Action:** Implement a loyalty program or CRM to nurture high-value recurring customers. Offering exclusive discounts or early access to new products can further increase retention and average order value.

**Insight 3: Low-cost home decor products are the gateway.**
- **Note:** The top 10 products by revenue include items such as `PAPER CRAFT` and `PARTY BUNTING`, which generally have a low unit cost but high volume.